# 4. Prediction of Human Oral Fraction Absorbed

This notebook describes some exploratory data analysis and the construction of predictive models of human oral fraction absorbed. Decision tree classifiers and tree-based ensemble methods (AdaBoostClassifier, RandomForestClassifier, and GradientBoostingClassifier) were considered due to their established success in ADMET prediction tasks, their ability to perform explicit/implicit feature selection, and because they are universal function approximators. 

In [ ]:
# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Cheminformatics libraries
from descriptastorus.descriptors.DescriptorGenerator import MakeGenerator
import mordred
from mordred import Calculator, descriptors
from rdkit import Chem
from rdkit.Chem import Descriptors, inchi, MACCSkeys, AllChem, DataStructs, PandasTools, PyMol, Lipinski
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.PandasTools import LoadSDF
from rdkit.DataStructs import ConvertToNumpyArray

# Data manipulation and plotting libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches

# sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_validate, KFold, StratifiedKFold

# Other
import ast
import pickle
from scipy import spatial as sp

In [ ]:
# Read necessary .csv files
original_df_ = pd.read_csv("../csv_files/final_master_df.csv", index_col = 0)
actives_df = pd.read_csv("../csv_files/adding_chembl_to_actives.csv", index_col = 0)

# Isolating oral drugs
original_df = original_df_[["actives_in_dosage_form","Dosage Form", "Route", "Fa1", "Fa2", "Fa3", "Fa4", "F1", "F2", "F3", "F4", "Tmax1", "Tmax2", "Tmax3", "Tmax4", "Excipients_Final"]]
original_df.replace("Blank", np.nan, inplace = True)
intermediate_df = original_df[original_df["Route"]=="Oral"]

# Convert multiple columns into arrays in a single column so .explode() method can be used
intermediate_df["actives_in_dosage_form"] = intermediate_df["actives_in_dosage_form"].apply(lambda x: x.upper().strip().split(","))
intermediate_df["Fa"] = intermediate_df.apply(lambda row: [row["Fa1"], row["Fa2"], row["Fa3"], row["Fa4"]], axis=1)
intermediate_df["F"] = intermediate_df.apply(lambda row: [row["F1"], row["F2"], row["F3"], row["F4"]], axis=1)
intermediate_df["Tmax"] = intermediate_df.apply(lambda row: [row["Tmax1"], row["Tmax2"], row["Tmax3"], row["Tmax4"]], axis=1)
intermediate_df.drop(["Fa1", "Fa2", "Fa3", "Fa4", "F1", "F2", "F3", "F4", "Tmax1", "Tmax2", "Tmax3", "Tmax4"], axis=1, inplace=True)

# Use the .explode() method on multiple columns
def prepare_for_explode(row):
    length = len(row["actives_in_dosage_form"])
    row["Fa"] = row["Fa"][:length]
    row["F"] = row["F"][:length]
    row["Tmax"] = row["Tmax"][:length]
    return row
intermediate_df = intermediate_df.apply(prepare_for_explode, axis=1)
intermediate_df = intermediate_df.explode(["actives_in_dosage_form", "Fa", "F", "Tmax"])
intermediate_df["actives_in_dosage_form"] = intermediate_df["actives_in_dosage_form"].apply(lambda x: x.strip())
actives_df = actives_df[["active_PSS","pss_inchi", "IsomericSMILES", "ps_smiles", "ps_num_drugs", "p_smiles", "p_inchi","p_inchikey"]].drop_duplicates()
actives_df.rename(columns = {"active_PSS": "actives_in_dosage_form"}, inplace=True)

# Merge into one final DataFrame and convert strings to floats
final_df = pd.merge(intermediate_df, actives_df, how="left", on="actives_in_dosage_form")
final_df.drop_duplicates(inplace=True)
final_df["Fa"], final_df["F"], final_df["Tmax"] = final_df["Fa"].astype(float), final_df["F"].astype(float), final_df["Tmax"].astype(float)
final_df.reset_index(inplace=True)

# Remove multicomponent parent molecules
final_df["multicomponent"] = final_df["p_smiles"].apply(lambda x: "." in x)
filtered_df = final_df[final_df["multicomponent"] == False]
filtered_df_f = filtered_df[filtered_df["F"].notna()].drop_duplicates(subset = ["p_inchikey"])
filtered_df = filtered_df[filtered_df["Fa"].notna()].drop_duplicates(subset = ["p_inchikey"])


In [ ]:
# Generate dataframes for EDA plots - Route
route_counts = original_df["Route"].value_counts()
other_count = route_counts[route_counts < 11].sum()
route_counts = route_counts[route_counts >= 11]
route_counts["Other"] = other_count
oral_counts = original_df[original_df["Route"] == "Oral"]["Dosage Form"].value_counts()
other_dosages = oral_counts[oral_counts < 40].sum()
oral_counts = oral_counts[oral_counts >= 40]
oral_counts["Other"] = other_dosages.sum()
route_counts_df = route_counts.replace("Oral", 0).to_frame()
for form, count in oral_counts[::-1].items():
    route_counts_df[form] = 0
route_counts_df["count"]["Oral"] = 0
route_counts_df["Tablet"]["Oral"] = oral_counts["Tablet"]
route_counts_df["Hard Capsule"]["Oral"] = oral_counts["Hard Capsule"]
route_counts_df["Solution"]["Oral"] = oral_counts["Solution"]
route_counts_df["Other"]["Oral"] = oral_counts["Other"]

# Fa and F
bin_edges_fa = np.linspace(min(filtered_df["Fa"]), max(filtered_df["Fa"]), 21)
bin_edges_f = np.linspace(min(filtered_df_f["F"]), max(filtered_df_f["F"]), 21)
fa_low = filtered_df[filtered_df["Fa"] < 0.8]["Fa"]
fa_high = filtered_df[filtered_df["Fa"] >= 0.8]["Fa"]
f_low = filtered_df_f[filtered_df_f["F"] < 0.8]["F"]
f_high = filtered_df_f[filtered_df_f["F"] >= 0.8]["F"]

# ATC Codes
atc_codes = original_df_[["ATC code", "Route"]]
atc_codes = atc_codes.dropna()  # Removing agents not assigned an ATC code
atc_codes = atc_codes[atc_codes["ATC code"].str.len() < 13] # Removing "not yet assigned"
atc_codes["ATC code"] = atc_codes["ATC code"].str[0]
atc_codes["Route"] = atc_codes["Route"] == "Oral"
atc_codes["Route"] = atc_codes["Route"].map({True: "Oral", False: "Not Oral"})
atc_codes = atc_codes.groupby(["ATC code", "Route"]).size().unstack(fill_value=0)
atc_codes.columns.name = None  
atc_codes.columns = ["Not oral", "Oral"]
atc_codes["Total"] = atc_codes["Not oral"] + atc_codes["Oral"]
atc_codes.sort_values(by="Total", inplace=True)
atc_codes["Category"]  = atc_codes.apply(lambda row: "Other" if row["Total"] < 45 else row.name, axis=1)
atc_codes = atc_codes.groupby("Category").sum()
atc_codes = atc_codes.sort_values("Total", ascending=False)
if "Other" in atc_codes.index:
    other_row = atc_codes.loc[["Other"]]  # Extract the "Other" row
    atc_codes = atc_codes.drop("Other")  # Drop the "Other" row from the main DataFrame
    atc_codes = pd.concat([atc_codes, other_row]) 
mapping = {
    "L": "Antineoplastics and\nimmunomodulators",
    "N": "Nervous system",
    "J": "Antiinfectives", 
    "A": "Alimentary tract\nand metabolism",
    "C": "Cardiovascular",
    "B": "Blood and blood\nforming organs",  
    "R": "Respiratory",
    "G": "Genito-urinary system/\nsex hormones",
    "Other": "Other ATC Codes"
}
atc_codes = atc_codes.rename(index=mapping)

In [ ]:
# Define gridspec layout
fig = plt.figure(figsize=(17, 10))
gs = gridspec.GridSpec(2, 3, width_ratios=[1, 1, 1], height_ratios=[1, 1], wspace=0.4, hspace=0.4)

# Plot 1: Fraction Absorbed
ax1 = fig.add_subplot(gs[0, 2])
ax1.hist(fa_low, bins=bin_edges_fa, color="C0", edgecolor="black", label=f"F$_{{a}}$ < 0.8 (n={len(fa_low)})")
ax1.hist(fa_high, bins=bin_edges_fa, color="C1", edgecolor="black", label=f"F$_{{a}}$ ≥ 0.8 (n={len(fa_high)})")
ax1.set_xlabel("Fraction Absorbed", fontsize=13)
legend_patches = [
    mpatches.Patch(facecolor="C0", label=f"F$_{{a}}$ < 0.8 (n={len(fa_low)})"),
    mpatches.Patch(facecolor="C1", label=f"F$_{{a}}$ ≥ 0.8 (n={len(fa_high)})")
]
ax1.legend(handles=legend_patches, fontsize=13)
ax1.set_ylabel("Frequency", fontsize=13)
ax1.spines["right"].set_visible(False)
ax1.spines["top"].set_visible(False)
ax1.text(-0.25, 1, "B", transform=ax1.transAxes, fontsize=25, fontweight="bold")
ax1.tick_params(axis="x", labelsize=11)
ax1.tick_params(axis="y", labelsize=11)

# Plot 2: Oral Bioavailability
ax2 = fig.add_subplot(gs[1, 2])
ax2.hist(filtered_df_f["F"], bins=bin_edges_f, color="C0", edgecolor="black")
ax2.set_xlabel("Oral Bioavailability", fontsize=13)
ax2.set_ylabel("Frequency", fontsize=13)
ax2.spines["right"].set_visible(False)
ax2.spines["top"].set_visible(False)
ax2.text(-0.25, 1, "D", transform=ax2.transAxes, fontsize=25, fontweight="bold")
ax2.tick_params(axis="x", labelsize=11)
ax2.tick_params(axis="y", labelsize=11)

# Plot 3: Route of Administration
legend_colors = ["C4", "C3", "C2", "C1"]
legend_labels = [f"Tablet* (n={route_counts_df["Tablet"].sum()})", 
                 f"Hard Capsule* (n={route_counts_df["Hard Capsule"].sum()})", 
                f"Solution (n={route_counts_df["Solution"].sum()})", 
                f"Other (n={route_counts_df["Other"].sum()})"]
legend_handles = [mpatches.Patch(color=color, label=label) for color, label in zip(legend_colors, legend_labels)]
ax3 = fig.add_subplot(gs[0, :2])
route_counts_df[::-1].plot(kind="barh", stacked=True, edgecolor="black", ax=ax3, legend=False)
ax3.set_ylabel("Route", fontsize=13)
ax3.set_xlabel("Frequency", fontsize=13)
ax3.grid(axis="x", alpha = 0.5)
row_sums = route_counts_df[::-1].sum(axis=1)  
bar_positions = range(len(row_sums)) 
for y, total_count in zip(bar_positions, row_sums):
    ax3.text(
        total_count + 5,
        y,
        str(total_count), 
        va="center", 
        ha="left", 
        color="black",
        fontsize=13
    )
ax3.spines["right"].set_visible(False)
ax3.spines["top"].set_visible(False)
ax3.legend(handles = legend_handles, title="Oral Dosage Forms:", loc=(0.65, 0.2), fontsize=13,  title_fontsize=13)
ax3.annotate("*Excludes dosage forms specified\nto have modified release ",  (593, 0.21), fontstyle="italic", fontsize=13)
ax3.text(-0.25, 1, "A", transform=ax3.transAxes, fontsize=25, fontweight="bold")
ax3.tick_params(axis="x", labelsize=11)
ax3.tick_params(axis="y", labelsize=11)

# Plot 4: Route of Administration 2
ax4 = fig.add_subplot(gs[1, :2])
atc_codes[["Oral", "Not oral"]][::-1].plot(kind="barh", stacked=True,  edgecolor="black", ax=ax4, legend=False)
ax4.set_ylabel("ATC Level One\n", fontsize=13)
ax4.set_xlabel("Frequency", fontsize=13)
ax4.spines["right"].set_visible(False)
ax4.spines["top"].set_visible(False)
ax4.grid(axis="x", alpha = 0.5)
bars = ax4.patches
legend_patches = [
    mpatches.Patch(facecolor="C0", label=f"Oral Route"),
    mpatches.Patch(facecolor="C1", label=f"Not Oral Route")
]
row_sums = atc_codes[["Oral", "Not oral"]][::-1].sum(axis=1)  
bar_positions = range(len(row_sums)) 
for y, total_count in zip(bar_positions, row_sums):
    ax4.text(
        total_count + 2,
        y,
        str(total_count), 
        va="center", 
        ha="left", 
        color="black",
        fontsize=13
    )
ax4.legend(handles = legend_patches, title="Route of Administration:", loc=(0.65, 0.2), fontsize=13, title_fontsize=13)
ax4.tick_params(axis="y", labelsize=8)
ax4.text(-0.25, 1, "C", transform=ax4.transAxes, fontsize=25, fontweight="bold")
ax4.tick_params(axis="x", labelsize=11)
ax4.tick_params(axis="y", labelsize=11)

plt.tight_layout()
plt.savefig("../figures/eda.pdf")
plt.show()

In [ ]:
# Three-dimensional feature set - mordred with RDKit default embedding
def get_embedded_mol(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        mol = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol)
        AllChem.MMFFOptimizeMolecule(mol)
        return mol
    except:
        return "Error"
filtered_df["Mol"] = filtered_df["p_smiles"].apply(get_embedded_mol)
filtered_df.reset_index(drop=True, inplace=True)

In [ ]:
# Calculating all mordered descriptors
# Kernel dies if done in the same cell as optimisation
calc = Calculator(descriptors, ignore_3D=False)
all_desc = calc.pandas(filtered_df["Mol"])
all_desc.dropna(axis=1, inplace=True)
all_desc = all_desc.select_dtypes(exclude=["object"])
X_mordred = all_desc.replace([np.inf, -np.inf], np.nan).dropna(axis=1, how="any")

In [ ]:
# One-dimensional feature set - MACCS Keys
maccs_keys_list = []
for smiles in filtered_df["p_smiles"]:
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        maccs_keys = MACCSkeys.GenMACCSKeys(mol)
        maccs_keys_list.append(list(maccs_keys.ToBitString()))
X_MACCS = pd.DataFrame(maccs_keys_list, columns=["MACCS_" + str(i) for i in range(167)])
X_MACCS.replace({"0": False, "1": True}, inplace=True)


# Two-dimensional feature set - RDKit2D
generator = MakeGenerator(("RDKit2D",))
X_RDKit2D = pd.DataFrame([generator.process(smiles) for smiles in filtered_df["p_smiles"]])
X_RDKit2D.set_axis([name for name, _ in generator.GetColumns()], axis=1)

# ECFP Fingerprints
ecfp6_list = []
for smiles in filtered_df["p_smiles"]:
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        ecfp6 = AllChem.GetMorganFingerprintAsBitVect(mol, 3, nBits=2048)  # ECFP6 with radius 3 and 2048 bits
        ecfp6_list.append(list(ecfp6.ToBitString()))
X_ECFP6 = pd.DataFrame(ecfp6_list, columns=["ECFP6_" + str(i) for i in range(2048)])
X_ECFP6.replace({"0": False, "1": True}, inplace=True)

# Lipinski Descriptors
def encode_molecule(SMILES_string):
    mol = Chem.MolFromSmiles(SMILES_string)
    mol_encoding = [Lipinski.FractionCSP3(mol), Lipinski.HeavyAtomCount(mol), Lipinski.NHOHCount(mol),
                    Lipinski.NOCount(mol), Lipinski.NumAliphaticCarbocycles(mol), Lipinski.NumAliphaticHeterocycles(mol),
                    Lipinski.NumAliphaticRings(mol), Lipinski.NumAromaticCarbocycles(mol), Lipinski.NumAromaticHeterocycles(mol),
                    Lipinski.NumAromaticRings(mol), Lipinski.NumHAcceptors(mol), Lipinski.NumHDonors(mol),
                    Lipinski.NumHeteroatoms(mol), Lipinski.NumRotatableBonds(mol), Lipinski.NumSaturatedCarbocycles(mol),
                    Lipinski.NumSaturatedHeterocycles(mol), Lipinski.NumSaturatedRings(mol), Lipinski.RingCount(mol)]
    return mol_encoding
data1 = filtered_df["p_smiles"]
feature_list = []
for smile in data1:
    feature_list.append(encode_molecule(smile))
data2 = np.array(feature_list)
X_lipinski = pd.DataFrame(data2)

In [ ]:
skf = StratifiedKFold(n_splits=10, random_state=0, shuffle=True)
classifiers = {
    "Random Forest Classifier": {
        "model": RandomForestClassifier(random_state=0),
        "param_grid": {
            "n_estimators": [25, 50, 100],
            "max_depth": [4, 8],
            "min_samples_split": [2, 3, 5],
            "min_samples_leaf": [1, 2, 4],
            "max_features": [None, "sqrt"] # 108 combinations
        }
      },    
    "AdaBoost Classifier": {
        "model": AdaBoostClassifier(random_state=0, algorithm="SAMME"),
        "param_grid": {
            "n_estimators": [25, 50, 100],
            "learning_rate": [0.01, 0.1, 0.5], # 9 combiantions
        }
   },
    "Gradient Boosting Machine": {
        "model": GradientBoostingClassifier(random_state=0),
        "param_grid": {
            "n_estimators": [25, 50, 100],
            "learning_rate": [0.01, 0.1, 0.5],
            "max_depth": [2, 4, 8] # 27 combinations
       }
    },
    "Decision Tree Classifier": {
    "model": DecisionTreeClassifier(random_state=0),
    "param_grid": {
        "criterion": ["gini", "entropy"],
        "max_depth": [2, 4, 8],
        "min_samples_split": [2, 4, 8],
        "min_samples_leaf": [1, 2, 4],
        "max_features": [None, "sqrt"], # 108 combinations
        }
    }
}

In [ ]:
# Defining X and Y
filtered_df["0.8_cutoff"] = filtered_df["Fa"].apply(lambda x: True if x >= 0.8 else False)
y = filtered_df["0.8_cutoff"]
X_list = [X_MACCS, X_ECFP6, X_lipinski, X_RDKit2D, X_mordred]
dataset_names = ["MACCS", "ECFP6", "Lipinski", "RDKit2D", "Mordred"]
results_list = []

# Grid search of hyperparameters, followed by refitting on all the data for the model which should the highest cross-validation balanced accuracy
# and obtaining final train and test perfomance metrics. Note that the best model was not selected on the basis of test set performance, but
# cross-validation performance. 

for X, dataset in zip(X_list, dataset_names):
    print(f"Descriptor set: {dataset}")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0) 
    for name, classifier in classifiers.items():
        print(f"Algorithm name: {name}")
        model = classifier["model"]
        param_grid = classifier["param_grid"]
        grid_search = GridSearchCV(model, param_grid, scoring="balanced_accuracy", cv=skf, n_jobs=-1) # Parallel to reduce computation time
        grid_search.fit(X_train, y_train)
        cv_results = pd.DataFrame(grid_search.cv_results_)
        best_model = grid_search.best_estimator_
        best_model.fit(X_train, y_train)
        y_pred_test = best_model.predict(X_test)
        y_pred_train = best_model.predict(X_train)
        top_results = cv_results.sort_values("mean_test_score", ascending=False).head(1)
        top_results["descriptor_set"] = dataset
        top_results["classifier"] = name
        top_results["train_ba"] = balanced_accuracy_score(y_train, y_pred_train)
        top_results["test_ba"] = balanced_accuracy_score(y_test, y_pred_test)
        results_list.append(top_results)

final_cv_df = pd.concat(results_list, ignore_index=True)
final_cv_df.sort_values(by=["mean_test_score"], ascending=[False], inplace=True) # Best model should be selected on basis of CV performance
final_cv_df.drop_duplicates(subset="descriptor_set", keep="first", inplace=True) # Only considering best model for each descriptor set
final_cv_df

In [ ]:
final_cv_df.to_csv("../results/cross_validation_results.csv")

In [ ]:
# For Table 2. Mordred may be variable between runs due to embedding.
final_cv_df[["descriptor_set", "classifier", "mean_test_score", "std_test_score", "train_ba", "test_ba"]].round(3)
# Generating LaTeX for manuscript
# print(final_cv_df[["descriptor_set", "classifier", "mean_test_score", "std_test_score", "train_ba", "test_ba"]].round(3).to_latex(index=False))

In [ ]:
# Additional info for tree classifier

X_train, X_test, y_train, y_test = train_test_split(X_RDKit2D, y, test_size=0.2, stratify=y, random_state=0) 
model = DecisionTreeClassifier(random_state=0)

param_grid = {
        "criterion": ["entropy"],
        "max_depth": [8],
        "min_samples_split": [8],
        "min_samples_leaf": [1],
        "max_features": ["sqrt"],
        }

grid_search = GridSearchCV(model, param_grid, scoring="balanced_accuracy", cv=skf, n_jobs=-1) # Parallel to reduce computation time
grid_search.fit(X_train, y_train)
tree_cv_results = pd.DataFrame(grid_search.cv_results_)
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)
y_pred_test = best_model.predict(X_test)
y_pred_train = best_model.predict(X_train)

test_accuracy = balanced_accuracy_score(y_test, y_pred_test)
print("Test Accuracy:", test_accuracy)

train_accuracy = balanced_accuracy_score(y_train, y_pred_train)
print("Train Accuracy:", train_accuracy)

conf_matrix = confusion_matrix(y_test, y_pred_test)
print(conf_matrix)

TN, FP = conf_matrix[0]
FN, TP = conf_matrix[1]

SE = TP / (TP + FN)
SP = TN / (TN + FP)
PPV = TP / (TP + FP)
NPV = TN / (TN + FN)

print(f"Sensitivity (SE): {SE:.2f}")
print(f"Specificity (SP): {SP:.2f}")
print(f"Positive Predictive Value (PPV): {PPV:.2f}")
print(f"Negative Predictive Value (NPV): {NPV:.2f}")

In [ ]:
# Saving best model
tree_text = export_text(best_model, feature_names=generator.GetColumns())
with open("../results/decision_tree_description.txt", "w") as f:
    f.write(tree_text)
with open("../results/best_model.pkl", "wb") as f:
    pickle.dump(best_model, f)

plt.figure(figsize=(20,10))
plot_tree(best_model, filled=True, feature_names=generator.GetColumns())
plt.show()

In [ ]:
class_assignments = pd.concat([X_RDKit2D, y], axis=1)
class_assignments[["drug_substance", "p_smiles"]] = filtered_df[["actives_in_dosage_form", "p_smiles"]]
class_assignments["Set"] = "Train"
class_assignments.loc[y_test.index, "Set"] = "Test"
class_assignments["Pred"] = None
class_assignments.loc[y_train.index, "Pred"] = y_pred_train
class_assignments.loc[y_test.index, "Pred"] = y_pred_test
class_assignments.to_csv("../results/train_test_split_results.csv")

In [ ]:
# Modified slightly from code made available by https://github.com/molecularmodelinglab/MODI/tree/main
# Used in Rath et al. JMedChem doi/10.1021/acs.jmedchem.3c02446

def nearest_neighbors(reference, query, k=1, self_query=False):
    tree = sp.KDTree(reference)
    if self_query:
        k = [x+2 for x in range(k)]
    else:
        k = [x+1 for x in range(k)]
    _, i = tree.query(query, k=k, workers=-1)
    return i

def modi(data, labels):
    classes = np.unique(labels)
    k = classes.shape[0]
    nn_idx = nearest_neighbors(data, data, k=1, self_query=True)
    nn_labels = labels[nn_idx]
    modi_value = 0
    class_contrib = []
    for c in classes:
        c_arr = np.where(labels == c)[0]
        c_labels = labels[c_arr]
        c_nn_labels = nn_labels[c_arr].flatten()
        modi_value += np.sum(c_labels == c_nn_labels) / c_arr.shape[0]
        class_contrib.append((c, np.sum(c_labels == c_nn_labels) / c_arr.shape[0]))
    return (k ** -1) * modi_value

In [ ]:
# Scaling is important as modelability index is based on nearest neighbour in Euclidean distance. 
# Scaling should in theory not affect tree performance as it is based on entropy/mutual info/
# some other metric of performance.

scaled_rdkit = StandardScaler().fit_transform(X_RDKit2D)
scaled_mordred = StandardScaler().fit_transform(X_mordred)
scaled_lipinski = StandardScaler().fit_transform(X_lipinski)
y_numpy = y.to_numpy()

print(f"MACCS: {modi(X_MACCS.to_numpy(), y_numpy)}")
print(f"ECFP6: {modi(X_ECFP6.to_numpy(), y_numpy)}")
print(f"RDKit2D: {modi(scaled_rdkit, y_numpy)}")
print(f"Lipinski: {modi(scaled_lipinski, y_numpy)}")
print(f"Mordred: {modi(scaled_mordred, y_numpy)}")

In [ ]:
feature_names=generator.GetColumns()
X_RDKit2D.columns = [ x for x, _ in feature_names]
spearman_corr = abs(X_RDKit2D.corr(method="spearman")["Chi0n"])
spearman_corr[spearman_corr > 0.9]